# DNN Entity Embedding

code inhttps://github.com/kitayama1234/Pytorch-Entity-Embedding-DNN-Regressor

In [ ]:
!git clone https://github.com/GalGoldstein/box_office_revenue_prediction.git

Cloning into 'box_office_revenue_prediction'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 59 (delta 33), reused 42 (delta 18), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [ ]:
!pip install git+https://github.com/yotammarton/TransformDF2Numpy

  Cloning https://github.com/yotammarton/TransformDF2Numpy to /tmp/pip-req-build-o8n3dhy8
  Running command git clone -q https://github.com/yotammarton/TransformDF2Numpy /tmp/pip-req-build-o8n3dhy8
  Created wheel for df2numpy: filename=df2numpy-1.1.0-cp37-none-any.whl size=9517 sha256=c030a56360ced5dfcbdadf8ad45b022067332a008da9d2f7af2db6318f1b97d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-vh1rn1fm/wheels/12/93/3a/4c7ff164f313f7106f73d18400b555ba5156df98447050d244
Successfully built df2numpy


In [ ]:
!git clone https://github.com/kitayama1234/Pytorch-Entity-Embedding-DNN-Regressor.git

Cloning into 'Pytorch-Entity-Embedding-DNN-Regressor'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (162/162), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 162 (delta 71), reused 109 (delta 39), pack-reused 0
Receiving objects: 100% (162/162), 7.72 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [ ]:
!mkdir box_office_revenue_prediction/hw1_data
!unzip box_office_revenue_prediction/hw1_data.zip -d box_office_revenue_prediction/hw1_data

Archive:  box_office_revenue_prediction/hw1_data.zip
  inflating: box_office_revenue_prediction/hw1_data/test.tsv  
  inflating: box_office_revenue_prediction/hw1_data/train.tsv  
  inflating: box_office_revenue_prediction/hw1_data/predict.py  


In [ ]:
!pip install catboost

     |████████████████████████████████| 67.3MB 90kB/s 


In [ ]:
!pip install pycaret

     |████████████████████████████████| 266kB 4.4MB/s 
     |████████████████████████████████| 276kB 32.0MB/s 
     |████████████████████████████████| 6.8MB 32.4MB/s 
     |████████████████████████████████| 61kB 5.0MB/s 
     |████████████████████████████████| 1.7MB 34.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0MB 31.5MB/s 
     |████████████████████████████████| 112kB 38.2MB/s 
     |████████████████████████████████| 245kB 36.2MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 14.2MB 293kB/s 
     |████████████████████████████████| 1.4MB 33.5MB/s 
     |████████████████████████████████| 174kB 46.7MB/s 
     |████████████████████████████████| 3.1MB 28.9MB/s 
     |████████████████████████████████| 1.1MB 25.0MB/s 
     |████████████████████████████████| 61kB 6.6MB/s

In [ ]:
import os
import numpy as np

In [ ]:
cwd = os.getcwd()

In [ ]:
os.chdir('/content/box_office_revenue_prediction')
from preprocess_data import load_data_transform2DFNumpy

In [ ]:
(X_train, y_train), (X_test, y_test), num_categoricals, trans = load_data_transform2DFNumpy(scale_popularity=True, return_trans=True)
original_y_test = np.array(y_test)

Starting to fit a transformer of TransformDF2Numpy.
Categories thresholded: (column: 'belongs_to_collection'), (valid categories: 25, dropped categories: 687)
Numerical NaNs filled with alternative value: (column: 'budget'), (filled rows: 1487, value: 30047340.334764)
Categories thresholded: (column: 'genres'), (valid categories: 19, dropped categories: 1)
Categories thresholded: (column: 'original_language'), (valid categories: 24, dropped categories: 19)
Categories thresholded: (column: 'production_companies'), (valid categories: 185, dropped categories: 2243)
Numerical NaNs filled with alternative value: (column: 'runtime'), (filled rows: 16, value: 108.276784)
Categories thresholded: (column: 'cast_1'), (valid categories: 319, dropped categories: 2238)
Categories thresholded: (column: 'cast_2'), (valid categories: 250, dropped categories: 3023)
Categories thresholded: (column: 'cast_3'), (valid categories: 145, dropped categories: 3581)
Categories thresholded: (column: 'cast_4'), (

In [ ]:
y_train, y_test = np.log1p(y_train), np.log1p(y_test)

In [ ]:
os.chdir('/content/Pytorch-Entity-Embedding-DNN-Regressor')
import numpy as np
import pandas as pd
from demo_pre_processor import MyPreprocessor
from df2numpy import TransformDF2Numpy   # https://github.com/kitayama1234/TransformDF2Numpy
from model import CategoricalDnn
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

torch.manual_seed(0)

In [ ]:
df_train, df_val = train_test_split(np.concatenate((X_train, y_train.reshape(-1, 1)), axis=1), test_size=0.1, random_state=10)

In [ ]:
len(df_train), len(df_val)

(4693, 522)

In [ ]:
X_train, y_train = df_train[:,:-1], df_train[:,-1]
X_val, y_val = df_val[:,:-1], df_val[:,-1]

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((4693, 20), (4693,), (522, 20), (522,))

In [ ]:
X_train = torch.from_numpy(X_train).float()
X_val = torch.from_numpy(X_val).float()
y_train = torch.from_numpy(y_train.reshape(-1, 1)).float()
y_val = torch.from_numpy(y_val.reshape(-1, 1)).float()
X_test =torch.from_numpy(X_test).float()
original_y_test = torch.from_numpy(original_y_test.reshape(-1, 1)).float()

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32)

test_dataset = TensorDataset(X_test, original_y_test)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
def make_embedding_construction(trans, compression_factor, max_embedded_features):
    embedding_construction = []
    total_embedded_features = 0
    for uniques in trans.nuniques():
        target = int(uniques * compression_factor)
        if target < 2:
            target = 2
        elif target > max_embedded_features:
            target = max_embedded_features
        embedding_construction.append([uniques, target])
        total_embedded_features += target
    total_features = total_embedded_features + trans.num_numericals
    return embedding_construction


emb_const = make_embedding_construction(trans, 1/4, 120)

model = CategoricalDnn(categorical_nuniques_to_dims=emb_const,
                                          num_numerical_features=trans.num_numericals,
                                          fc_layers_construction=[100, 70],
                                          dropout_probability=0.1)

print(model)

CategoricalDnn(
  (embedding_layer_list): ModuleList(
    (0): Sequential(
      (0): Embedding(26, 6)
      (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Embedding(20, 5)
      (1): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Embedding(25, 6)
      (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (3): Sequential(
      (0): Embedding(186, 46)
      (1): BatchNorm1d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (4): Sequential(
      (0): Embedding(320, 80)
      (1): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (5): Sequential(
      (0): Embedding(251, 62)
      (1): Bat

In [ ]:
import torch.nn as nn
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
criterion = RMSLELoss()

In [ ]:
def rmsle(preds, true):
    """
    Compute the Root Mean Squared Log Error for predictions `preds` and targets `true`
    :param preds: predictions
    :param true: targets
    :return: RMSLE score
    """
    return np.sqrt(np.square(np.log(true + 1) - np.log(preds + 1)).mean())

# evaluating function for validation dataset
def evaluation(model, loader, test=False):
    predicted = None
    true_y = None
    model.eval()
    for i, (batch_test_x, batch_test_y) in enumerate(loader):
        batch_predicted = model(batch_test_x)
        batch_predicted = batch_predicted
        if i == 0:
            true_y = batch_test_y
            predicted = batch_predicted
        else:
            true_y = torch.cat([true_y, batch_test_y], axis=0)
            predicted = torch.cat([predicted, batch_predicted], axis=0)
    if test:
      loss = criterion(true_y, predicted)
      true_y = true_y.detach().numpy()
      predicted = np.expm1(predicted.detach().numpy())
      return loss, predicted, true_y, rmsle(predicted, true_y) 
    return criterion(true_y, predicted), predicted, true_y


# learning rate scheduler
def lr_schedule(e):
    if e in [16, 24, 36, 40, 44, 48, 52, 56, 60, 66, 69]:
        print("lr updated")
        optimizer.param_groups[0]["lr"] *= 0.1


# training function
def train(epochs):
    loss_in_one_epoch = torch.zeros(train_loader.__len__())
    for epoch in range(epochs):
        model.train()
        lr_schedule(epoch)
        for batch_index, (batch_x, batch_y) in enumerate(train_loader):
            out = model(batch_x)
            loss_temp = criterion(out, batch_y)
            optimizer.zero_grad()
            loss_temp.backward()
            optimizer.step()
            loss_in_one_epoch[batch_index] = loss_temp
        if epoch % 5 == 0:
            loss = loss_in_one_epoch.mean()
            loss_val = evaluation(model, val_loader)[0]
            eval_test = evaluation(model, test_loader, test=True)
            loss_test = eval_test[0]
            print("epoch: ", epoch, ", train loss: ", float(loss), end="")
            print(", val loss: ", float(loss_val), end="")
            print(", test rmsle: ", eval_test[3])

In [ ]:
train(40)

epoch:  0 , train loss:  1.9508198499679565, val loss:  1.7460912466049194, test rmsle:  14.373146
epoch:  5 , train loss:  0.626315712928772, val loss:  0.5519259572029114, test rmsle:  7.4624877
epoch:  10 , train loss:  0.2255314290523529, val loss:  0.24714024364948273, test rmsle:  2.7727296
epoch:  15 , train loss:  0.19167210161685944, val loss:  0.24364639818668365, test rmsle:  2.4014277
epoch:  20 , train loss:  0.18200941383838654, val loss:  0.24691011011600494, test rmsle:  2.4010797
lr updated
epoch:  25 , train loss:  0.17139171063899994, val loss:  0.24307163059711456, test rmsle:  2.4086144
epoch:  30 , train loss:  0.16362197697162628, val loss:  0.24397669732570648, test rmsle:  2.3964334
lr updated
epoch:  35 , train loss:  0.16223320364952087, val loss:  0.2443883866071701, test rmsle:  2.3960288
